In [1]:
import pandas as pd


### הפונקציה 

In [2]:
def Create_Normalized_File(input_file, columns_to_normalize, LSOA_Input):
    input_file[columns_to_normalize] = input_file[columns_to_normalize].replace(',', '', regex=True).astype(float)

    # קריאת קובץ CSV ויצירת DataFrame עבור העמודות הרצויות של PostCode
    post_code_file_path = r"C:/Users/almog/Almog_codes/Anaconda/Code_in_Python/Geo_of_Health_Final_Project/MSOA_AND_LSOA/LSOA_From_web_01.csv"
    PostCode_df = pd.read_csv(post_code_file_path)
    PostCode_columns_df = PostCode_df[['PRACTICE_CODE', 'ORG_NAME', 'LSOA_CODE', 'All Patients']]
    PostCode_columns_df = PostCode_columns_df.rename(columns={'All Patients': 'All_Patients_per_LSOA'})
    PostCode_columns_df = PostCode_columns_df.sort_values(by=['PRACTICE_CODE', 'LSOA_CODE']).copy()
    PostCode_columns_df['tot_patients_per_PRACTICE_CODE'] = PostCode_columns_df.groupby('PRACTICE_CODE')['All_Patients_per_LSOA'].transform('sum')
    PostCode_columns_df['lsoa_patients_ratio'] = PostCode_columns_df['All_Patients_per_LSOA'] / PostCode_columns_df['tot_patients_per_PRACTICE_CODE']

    # מיזוג ה DataFrames PostCode_columns_df ו-input_file
    merged_df = pd.merge(PostCode_columns_df, input_file, left_on='LSOA_CODE', right_on=LSOA_Input, how='left')

    # מחיקת העמודה 'LSOA code (2011)' מה DataFrame שלנו
    merged_df = merged_df.drop(LSOA_Input, axis=1)

    # מיון ה-DataFrame לפי עמודות 'PRACTICE_CODE' ו-'LSOA_CODE' ויצירת עותק של ה-DataFrame
    merged_columns_df = merged_df.sort_values(by=['PRACTICE_CODE', 'LSOA_CODE']).copy()

    # יצירת רשימה ריקה שבה נאחסן את שמות העמודות החדשות שנוצרות
    new_column_names = []

    # לולאה שעוברת על רשימת העמודות הרצויות שנרצה ליצור מהן עמודות חדשות
    for column in columns_to_normalize:
        # יצירת שם חדש לעמודה בהתאם לשם המקורי והחישוב שנדרש
        new_column_name = f"lsoa_patients_ratio * {column.split(' Rank')[0]}"
        # יצירת העמודה החדשה על ידי כפל הערכים בעמודה 'lsoa_patients_ratio' ובעמודה המתאימה ב-'columns_to_normalize'
        merged_columns_df[new_column_name] = merged_columns_df['lsoa_patients_ratio'] * merged_columns_df[column]

        # חישוב הסכום המשוקלל עבור כל קוד רופא בעזרת קבוצת הקודים
        weighted_per_practice = merged_columns_df.groupby('PRACTICE_CODE').apply(lambda x: (x['lsoa_patients_ratio'] * x[column]).sum())
        
        # תנאי עבור כל ערך בסדרה weighted_per_practice
        #weighted_per_practice = weighted_per_practice.apply(lambda val: f"{val:.5f}" if val > 1 else f"{val:.5f}" if val >= -1 else val)
        weighted_per_practice = weighted_per_practice.apply(lambda val: int(round(val)) if val > 1 else f"{val:.2f}")


        # יצירת שם חדש לעמודה החדשה שמכילה את הסכום המשוקלל עבור כל קוד רופא
        new_column_name_practice = f'Practice_code_weighted_tot_of_{column.split(" Rank")[0]}'
        # הכנסת הערך של הסכום המשוקלל לכל רופא בעמודה החדשה
        merged_columns_df[new_column_name_practice] = merged_columns_df['PRACTICE_CODE'].map(weighted_per_practice)

        # הוספת שם העמודה החדשה לרשימה של העמודות החדשות
        new_column_names.append(new_column_name_practice)

    #  בניית קובץ סופי , ומחיקת LSOA
    Normalized_File = merged_columns_df[['PRACTICE_CODE', 'tot_patients_per_PRACTICE_CODE'] + new_column_names].drop_duplicates()

    return Normalized_File


# ‏‏PrivateGarden

In [3]:
PrivateGarden_df = pd.read_excel('PrivateGarden_01.xlsx')
MSOA_df =pd.read_csv("C:/Users/almog/Almog_codes/Anaconda/Code_in_Python/Geo_of_Health_Final_Project/MSOA_AND_LSOA\MSOA_filter.csv")


In [4]:
PrivateGarden_df.info()
MSOA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8483 entries, 0 to 8482
Data columns (total 26 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Country code                                                6793 non-null   object 
 1   Country name                                                6791 non-null   object 
 2   Region code                                                 6791 non-null   object 
 3   Region name                                                 6791 non-null   object 
 4   LAD code                                                    6791 non-null   object 
 5   LAD name                                                    6791 non-null   object 
 6   MSOA code                                                   6791 non-null   object 
 7   MSOA name                                                   6791 non-null   object 
 8 

In [5]:
MSOA_filter_df = MSOA_df[['lsoa11cd', 'msoa11cd']].sort_values(by=['msoa11cd', 'lsoa11cd'], ascending=True)
MSOA_filter_df.head(5)

,lsoa11cd,msoa11cd
0,E01000001,E02000001
1,E01000002,E02000001
2,E01000003,E02000001
3,E01000005,E02000001
31920,E01032739,E02000001


In [6]:
#sorting
PrivateGarden_sort_df = PrivateGarden_df.copy().sort_values(by='MSOA code', ascending=True)
PrivateGarden_sort_df.head(5)

,Country code,Country name,Region code,Region name,LAD code,LAD name,MSOA code,MSOA name,Houses - Address count,Houses - Adress with private outdoor space count,...,Flats - Private outdoor space total area (m2),Flats - Private outdoor space count,Flats - Percentage of adresses with private outdoor space,Flats - Average size of private outdoor space (m2),Flats - Average number of flats sharing a garden,Total - Address count,Total - Adress with private outdoor space count,Total - Private outdoor space total area (m2),Total - Percentage of adresses with private outdoor space,Total - Average size of private outdoor space (m2)
0,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,74.0,20.0,...,3294.4,13.4,0.070,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0
1,E92000001,England,E12000007,London,E09000002,Barking and Dagenham,E02000002,Barking and Dagenham 001,1728.0,1698.0,...,36880.3,165.4,0.481,222.98,3.6,2965.0,2293.0,1236691.1,0.773,663.7
2,E92000001,England,E12000007,London,E09000002,Barking and Dagenham,E02000003,Barking and Dagenham 002,2656.0,2634.0,...,69334.0,323.8,0.612,214.13,2.7,4071.0,3500.0,502488.6,0.860,169.9
3,E92000001,England,E12000007,London,E09000002,Barking and Dagenham,E02000004,Barking and Dagenham 003,2051.0,2037.0,...,16058.4,98.1,0.513,163.69,1.8,2390.0,2211.0,406017.6,0.925,190.2
4,E92000001,England,E12000007,London,E09000002,Barking and Dagenham,E02000005,Barking and Dagenham 004,3115.0,3089.0,...,27837.8,119.2,0.706,233.54,3.2,3660.0,3474.0,438159.0,0.949,136.6


In [7]:
PrivateGarden_merge_df = pd.merge(MSOA_filter_df, PrivateGarden_sort_df, left_on='msoa11cd', right_on='MSOA code', how='inner')
PrivateGarden_merge_df.head(5)

,lsoa11cd,msoa11cd,Country code,Country name,Region code,Region name,LAD code,LAD name,MSOA code,MSOA name,...,Flats - Private outdoor space total area (m2),Flats - Private outdoor space count,Flats - Percentage of adresses with private outdoor space,Flats - Average size of private outdoor space (m2),Flats - Average number of flats sharing a garden,Total - Address count,Total - Adress with private outdoor space count,Total - Private outdoor space total area (m2),Total - Percentage of adresses with private outdoor space,Total - Average size of private outdoor space (m2)
0,E01000001,E02000001,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,...,3294.4,13.4,0.07,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0
1,E01000002,E02000001,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,...,3294.4,13.4,0.07,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0
2,E01000003,E02000001,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,...,3294.4,13.4,0.07,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0
3,E01000005,E02000001,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,...,3294.4,13.4,0.07,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0
4,E01032739,E02000001,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,...,3294.4,13.4,0.07,245.85,39.9,7707.0,554.0,3674.5,0.072,110.0


In [8]:
PrivateGarden_merge_df.columns

Index(['lsoa11cd', 'msoa11cd', 'Country code', 'Country name', 'Region code',
       'Region name', 'LAD code', 'LAD name', 'MSOA code', 'MSOA name',
       'Houses - Address count',
       'Houses - Adress with private outdoor space count',
       'Houses - Private outdoor space total area (m2)',
       'Houses - Percentage of adresses with private outdoor space',
       'Houses - Average size of private outdoor space (m2)',
       'Houses - Median size of private outdoor space (m2)',
       'Flats - Address count',
       'Flats - Adress with private outdoor space count',
       'Flats - Private outdoor space total area (m2)',
       'Flats - Private outdoor space count',
       'Flats - Percentage of adresses with private outdoor space',
       'Flats - Average size of private outdoor space (m2)',
       'Flats - Average number of flats sharing a garden',
       'Total - Address count',
       'Total - Adress with private outdoor space count',
       'Total - Private outdoor space t

In [9]:
PrivateGarden_merge_df = PrivateGarden_merge_df.drop('msoa11cd', axis=1)


In [10]:
PrivateGarden_merge_df.to_csv('PrivateGarden_mergeOnly.csv', index=False)

## נרמול

### הכנה לנרמול

In [11]:
columns_to_normalize = ['Houses - Address count',
       'Houses - Adress with private outdoor space count',
       'Houses - Private outdoor space total area (m2)',
       'Houses - Percentage of adresses with private outdoor space',
       'Houses - Average size of private outdoor space (m2)',
       'Houses - Median size of private outdoor space (m2)',
       'Flats - Address count',
       'Flats - Adress with private outdoor space count',
       'Flats - Private outdoor space total area (m2)',
       'Flats - Private outdoor space count',
       'Flats - Percentage of adresses with private outdoor space',
       'Flats - Average size of private outdoor space (m2)',
       'Flats - Average number of flats sharing a garden',
       'Total - Address count',
       'Total - Adress with private outdoor space count',
       'Total - Private outdoor space total area (m2)',
       'Total - Percentage of adresses with private outdoor space',
       'Total - Average size of private outdoor space (m2)']

LSOA_CODE = 'lsoa11cd'

In [12]:
PrivateGarden_Normalized_FG_df = Create_Normalized_File(PrivateGarden_merge_df, columns_to_normalize, LSOA_CODE)


In [13]:
PrivateGarden_Normalized_FG_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7875 entries, 0 to 733708
Data columns (total 20 columns):
 #   Column                                                                                    Non-Null Count  Dtype 
---  ------                                                                                    --------------  ----- 
 0   PRACTICE_CODE                                                                             7875 non-null   object
 1   tot_patients_per_PRACTICE_CODE                                                            7875 non-null   int64 
 2   Practice_code_weighted_tot_of_Houses - Address count                                      7875 non-null   int64 
 3   Practice_code_weighted_tot_of_Houses - Adress with private outdoor space count            7875 non-null   int64 
 4   Practice_code_weighted_tot_of_Houses - Private outdoor space total area (m2)              7875 non-null   int64 
 5   Practice_code_weighted_tot_of_Houses - Percentage of adresses wit

In [14]:
PrivateGarden_Normalized_FG_df.head(5)

,PRACTICE_CODE,tot_patients_per_PRACTICE_CODE,Practice_code_weighted_tot_of_Houses - Address count,Practice_code_weighted_tot_of_Houses - Adress with private outdoor space count,Practice_code_weighted_tot_of_Houses - Private outdoor space total area (m2),Practice_code_weighted_tot_of_Houses - Percentage of adresses with private outdoor space,Practice_code_weighted_tot_of_Houses - Average size of private outdoor space (m2),Practice_code_weighted_tot_of_Houses - Median size of private outdoor space (m2),Practice_code_weighted_tot_of_Flats - Address count,Practice_code_weighted_tot_of_Flats - Adress with private outdoor space count,Practice_code_weighted_tot_of_Flats - Private outdoor space total area (m2),Practice_code_weighted_tot_of_Flats - Private outdoor space count,Practice_code_weighted_tot_of_Flats - Percentage of adresses with private outdoor space,Practice_code_weighted_tot_of_Flats - Average size of private outdoor space (m2),Practice_code_weighted_tot_of_Flats - Average number of flats sharing a garden,Practice_code_weighted_tot_of_Total - Address count,Practice_code_weighted_tot_of_Total - Adress with private outdoor space count,Practice_code_weighted_tot_of_Total - Private outdoor space total area (m2),Practice_code_weighted_tot_of_Total - Percentage of adresses with private outdoor space,Practice_code_weighted_tot_of_Total - Average size of private outdoor space (m2)
0,A81001,4175,3311,3238,558884,0.98,185,149,900,577,25437,124,0.66,193,4,4212,3815,584320,0.93,185
39,A81002,19617,3107,3062,587752,0.99,197,161,467,311,18873,104,0.70,200,3,3574,3374,606625,0.96,196
126,A81003,3470,3057,3014,477520,0.99,148,113,636,433,28461,189,0.72,213,4,3692,3447,505981,0.93,149
184,A81004,9078,2927,2891,492134,0.99,174,131,673,391,20520,136,0.70,199,3,3599,3281,512654,0.93,174
296,A81005,7918,2676,2575,681424,0.96,267,177,278,194,25640,80,0.74,681,6,2954,2769,707064,0.94,268


In [15]:
PrivateGarden_Normalized_FG_df.to_csv('PrivateGarden_Normalized_FG.csv', index=False)


# PublicGarden

In [34]:
PublicGarden_df = pd.read_excel('PublicGarden_01.xlsx', skiprows=2)
#MSOA_df =pd.read_csv("C:/Users/almog/Almog_codes/Anaconda/Code_in_Python/Geo_of_Health_Final_Project/MSOA_AND_LSOA\MSOA_filter.csv")

In [35]:
PublicGarden_df.info()
#MSOA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41397 entries, 0 to 41396
Data columns (total 32 columns):
 #   Column                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                           --------------  -----  
 0   Country code                                                                                                                     41396 non-null  object 
 1   Country name                                                                                                                     41394 non-null  object 
 2   Region code                                                                                                                      41394 non-null  object 
 3   Region name                                                                                     

In [36]:
#MSOA_filter_df = MSOA_df[['lsoa11cd', 'msoa11cd']].sort_values(by=['msoa11cd', 'lsoa11cd'], ascending=True)
#MSOA_filter_df.head(5)

In [37]:
#sorting
PublicGarden_sort_df = PublicGarden_df.copy().sort_values(by='LSOA code', ascending=True)
PublicGarden_sort_df.head(5)

,Country code,Country name,Region code,Region name,LAD code,LAD name,MSOA code,MSOA name,LSOA code,LSOA name,...,Parks and Playing Fields - Index of Multiple Deprivation decile (country specific),Parks and Playing Fields - Average distance to nearest Park Public Garden or Playing Field (m),Parks and Playing Fields - Average size of nearest Park Public Garden or Playing Field (m2),"Parks and Playing Fields - Average number of Parks Public Gardens or Playing Fields within 1,000 m radius","Parks and Playing Fields - Average combined size of Parks Public Gardens or Playing Fields within 1,000 m radius (m2)",Parks and Playing Fields - Number of postcodes within built up area,Parks and Playing Fields - Number of built up area postcodes within 300m of a Park Public Garden or Playing Field,Parks and Playing Fields - Number of built up area postcodes within 300m of a Park Public Garden or Playing Field (percentage),Parks and Playing Fields - Number of built up area postcodes within 900m of a Park Public Garden or Playing Field,Parks and Playing Fields - Number of built up area postcodes within 900m of a Park Public Garden or Playing Field (percentage)
0,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,E01000001,City of London 001A,...,9.0,183.60,2862.51,11.41,48963.87,54.0,51.0,0.94,54.0,1.0
1,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,E01000002,City of London 001B,...,10.0,258.92,12510.78,10.99,43326.18,84.0,52.0,0.62,84.0,1.0
2,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,E01000003,City of London 001C,...,5.0,153.09,3537.77,10.53,55801.24,34.0,34.0,1.00,34.0,1.0
3,E92000001,England,E12000007,London,E09000001,City of London,E02000001,City of London 001,E01000005,City of London 001E,...,3.0,164.26,603.00,12.67,57823.68,100.0,92.0,0.92,100.0,1.0
4,E92000001,England,E12000007,London,E09000002,Barking and Dagenham,E02000017,Barking and Dagenham 016,E01000006,Barking and Dagenham 016A,...,5.0,246.65,39927.22,5.29,504728.93,14.0,11.0,0.79,14.0,1.0


In [38]:
# לא צריך לאחד כי הפורמט הוא LSOA
#PublicGarden_merge_df = pd.merge(MSOA_filter_df, PublicGarden_sort_df, left_on='msoa11cd', right_on='MSOA code', how='inner')
#PublicGarden_merge_df.head(5)

#PublicGarden_merge_df.columns

#PublicGarden_merge_df = PublicGarden_merge_df.drop('msoa11cd', axis=1)

#PublicGarden_merge_df.to_csv('PublicGarden_mergeOnly.csv', index=False)

### נרמול

In [39]:
PublicGarden_sort_df.columns

Index(['Country code', 'Country name', 'Region code', 'Region name',
       'LAD code', 'LAD name', 'MSOA code', 'MSOA name', 'LSOA code',
       'LSOA name',
       'Parks only - Index of Multiple Deprivation rank (country specific)',
       'Parks only - Index of Multiple Deprivation decile (country specific)',
       'Parks only - Average distance to nearest Park or Public Garden (m)',
       'Parks only - Average size of nearest Park or Public Garden (m2)',
       'Parks only - Average number of Parks or Public Gardens within 1,000 m radius',
       'Parks only - Average combined size of Parks or Public Gardens within 1,000 m radius (m2)',
       'Parks only - Number of postcodes within built up area',
       'Parks only - Number of built up area postcodes  within 300m of a Park or Public Garden (percentage)',
       'Parks only - Number of built up area postcodes  within 300m of a Park or Public Garden',
       'Parks only - Number of built up area postcodes  within 900m of a Park

In [42]:
columns_to_normalize_2 = ['Parks only - Index of Multiple Deprivation rank (country specific)',
       'Parks only - Index of Multiple Deprivation decile (country specific)',
       'Parks only - Average distance to nearest Park or Public Garden (m)',
       'Parks only - Average size of nearest Park or Public Garden (m2)',
       'Parks only - Average number of Parks or Public Gardens within 1,000 m radius',
       'Parks only - Average combined size of Parks or Public Gardens within 1,000 m radius (m2)',
       'Parks only - Number of postcodes within built up area',
       'Parks only - Number of built up area postcodes  within 300m of a Park or Public Garden (percentage)',
       'Parks only - Number of built up area postcodes  within 300m of a Park or Public Garden',
       'Parks only - Number of built up area postcodes  within 900m of a Park or Public Garden (percentage)',
       'Parks only - Number of built up area postcodes  within 900m of a Park or Public Garden',
       'Parks and Playing Fields - Index of Multiple Deprivation rank (country specific)',
       'Parks and Playing Fields - Index of Multiple Deprivation decile (country specific)',
       'Parks and Playing Fields - Average distance to nearest Park Public Garden or Playing Field (m)',
       'Parks and Playing Fields - Average size of nearest Park Public Garden or Playing Field (m2)',
       'Parks and Playing Fields - Average number of  Parks Public Gardens or Playing Fields within 1,000 m radius',
       'Parks and Playing Fields - Average combined size of  Parks Public Gardens or Playing Fields within 1,000 m radius (m2)',
       'Parks and Playing Fields - Number of postcodes within built up area',
       'Parks and Playing Fields - Number of built up area postcodes  within 300m of a Park Public Garden or Playing Field',
       'Parks and Playing Fields - Number of built up area postcodes  within 300m of a Park Public Garden or Playing Field (percentage)',
       'Parks and Playing Fields - Number of built up area postcodes  within 900m of a Park Public Garden or Playing Field',
       'Parks and Playing Fields - Number of built up area postcodes  within 900m of a Park Public Garden or Playing Field (percentage)']


LSOA_CODE_2 = 'LSOA code'

In [43]:
PublicGarden_Normalized_FG_df = Create_Normalized_File(PublicGarden_sort_df, columns_to_normalize_2, LSOA_CODE_2)


In [44]:
PublicGarden_Normalized_FG_df.head(5)

,PRACTICE_CODE,tot_patients_per_PRACTICE_CODE,Practice_code_weighted_tot_of_Parks only - Index of Multiple Deprivation rank (country specific),Practice_code_weighted_tot_of_Parks only - Index of Multiple Deprivation decile (country specific),Practice_code_weighted_tot_of_Parks only - Average distance to nearest Park or Public Garden (m),Practice_code_weighted_tot_of_Parks only - Average size of nearest Park or Public Garden (m2),"Practice_code_weighted_tot_of_Parks only - Average number of Parks or Public Gardens within 1,000 m radius","Practice_code_weighted_tot_of_Parks only - Average combined size of Parks or Public Gardens within 1,000 m radius (m2)",Practice_code_weighted_tot_of_Parks only - Number of postcodes within built up area,Practice_code_weighted_tot_of_Parks only - Number of built up area postcodes within 300m of a Park or Public Garden (percentage),...,Practice_code_weighted_tot_of_Parks and Playing Fields - Index of Multiple Deprivation decile (country specific),Practice_code_weighted_tot_of_Parks and Playing Fields - Average distance to nearest Park Public Garden or Playing Field (m),Practice_code_weighted_tot_of_Parks and Playing Fields - Average size of nearest Park Public Garden or Playing Field (m2),"Practice_code_weighted_tot_of_Parks and Playing Fields - Average number of Parks Public Gardens or Playing Fields within 1,000 m radius","Practice_code_weighted_tot_of_Parks and Playing Fields - Average combined size of Parks Public Gardens or Playing Fields within 1,000 m radius (m2)",Practice_code_weighted_tot_of_Parks and Playing Fields - Number of postcodes within built up area,Practice_code_weighted_tot_of_Parks and Playing Fields - Number of built up area postcodes within 300m of a Park Public Garden or Playing Field,Practice_code_weighted_tot_of_Parks and Playing Fields - Number of built up area postcodes within 300m of a Park Public Garden or Playing Field (percentage),Practice_code_weighted_tot_of_Parks and Playing Fields - Number of built up area postcodes within 900m of a Park Public Garden or Playing Field,Practice_code_weighted_tot_of_Parks and Playing Fields - Number of built up area postcodes within 900m of a Park Public Garden or Playing Field (percentage)
0,A81001,4175,13864,5,660,156485,2,226989,59,0.23,...,5,486,87924,4,280781,59,20,0.35,54,0.86
39,A81002,19617,12571,4,675,259580,2,406606,50,0.21,...,4,455,109724,4,419448,50,18,0.38,45,0.88
126,A81003,3470,6417,3,684,84542,1,117318,51,0.20,...,3,503,65551,3,165705,51,17,0.36,43,0.86
184,A81004,9078,11599,4,939,98122,2,184542,51,0.26,...,4,454,87945,4,269883,51,23,0.45,45,0.87
296,A81005,7918,21247,7,624,30089,1,39532,44,0.19,...,7,317,41325,3,113379,44,26,0.60,44,0.99


In [45]:
PublicGarden_Normalized_FG_df.to_csv('PublicGarden_Normalized_FG.csv', index=False)
PublicGarden_Normalized_FG_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7875 entries, 0 to 733708
Data columns (total 24 columns):
 #   Column                                                                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                                                                         --------------  ----- 
 0   PRACTICE_CODE                                                                                                                                                  7875 non-null   object
 1   tot_patients_per_PRACTICE_CODE                                                                                                                                 7875 non-null   int64 
 2   Practice_code_weighted_tot_of_Parks only - Index of Multiple Deprivation rank (country specific)                                               